# Data Collection into CSVs

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import duckdb

In [2]:
#creates a years list of all the election years we have data for
years = np.linspace(2000, 2022, 12)
#initialize a dictionary
d = {}
#cycle through each of the years, adding the year as the key mapped to the dataframe for each year
for year in years:
    #skip rows for headers/US, parse thousands
    d[int(year)] = pd.read_csv('data/election_'+str(int(year))+'.csv', skiprows=[0,2] , thousands=",")
    #make a column for the year
    d[int(year)]['Year'] = int(year)

#make a list to store the dataframes
df_list = []
#for each year in the dictionary (for each year)
for year in d.items():
    #append the dataframe to the list
    df_list.append(year[1])

#concat all the dataframes together, ignoring index
election_year_data = pd.concat(df_list, ignore_index=True)

#print the shape as a sanity check, should number of rows should be 612 = 12*51
print(election_year_data.shape)

#use SQL to change certain column names and create 2 new columns for percentages
election_year_data = duckdb.sql("""SELECT "Unnamed: 0" AS State, Year, "Total Ballots Counted" AS TotalBallots,
                                "Voting-Eligible Population (VEP)" AS VotingEligiblePopulation, 
                                "Voting-Age Population (VAP)" AS VotingAgePopulation, 
                                TotalBallots/VotingEligiblePopulation AS PercentVotingEligibleVotes,
                                TotalBallots/VotingAgePopulation AS PercentVotingAgeVotes
                                FROM election_year_data""").df()

# #print the dataset to verify completion and accuracy
election_year_data


(612, 17)


,State,Year,TotalBallots,VotingEligiblePopulation,VotingAgePopulation,PercentVotingEligibleVotes,PercentVotingAgeVotes
0,Alabama,2000,NaN,3241682,3334576,NaN,NaN
1,Alaska,2000,287825.0,419111,440296,0.686751,0.653708
2,Arizona,2000,1559520.0,3357701,3816498,0.464461,0.408626
3,Arkansas,2000,NaN,1925961,2001774,NaN,NaN
4,California,2000,11142843.0,19685258,24867252,0.566050,0.448093
...,...,...,...,...,...,...,...
607,Virginia,2022,3021956.0,6348891,6836463,0.475982,0.442035
608,Washington,2022,3067686.0,5543001,6164761,0.553434,0.497616
609,West Virginia,2022,494753.0,1396169,1423003,0.354365,0.347682
610,Wisconsin,2022,2673154.0,4467396,4655496,0.598370,0.574193


In [3]:

#read in the data
education = pd.read_csv('data/education.csv', thousands=",")
#melt the data into long form
education = education.melt(id_vars=['Name'],value_name="PercentBachelors",var_name="Year")
#printing the shape should yield 459 rows
print(education.shape)
#print education to verify
education

(459, 3)


,Name,Year,PercentBachelors
0,Alabama,2006,21.1
1,Alaska,2006,26.9
2,Arizona,2006,25.5
3,Arkansas,2006,18.2
4,California,2006,29.0
...,...,...,...
454,Virginia,2022,42.2
455,Washington,2022,39.5
456,West Virginia,2022,24.8
457,Wisconsin,2022,33.2


In [4]:
#read in the csv
income = pd.read_csv('data/income.csv', thousands=",", skiprows=[1])
#melt the data into long form
income = income.melt(id_vars=['Name'],value_name="Income",var_name="Year")
#print the shape of the dataframe as a sanity check, should be 612 rows
print(income.shape)
#print the data frame to make sure everything looks right
income

(612, 3)


,Name,Year,Income
0,Alabama,2000,59070
1,Alaska,2000,88130
2,Arizona,2000,66340
3,Arkansas,2000,49520
4,California,2000,78070
...,...,...,...
607,Virginia,2022,88550
608,Washington,2022,92980
609,West Virginia,2022,54540
610,Wisconsin,2022,76240


In [5]:
#read in the CSV, skipping the row for the US as a whole
diversity = pd.read_csv('data/diversity.csv', thousands=",", skiprows=[1])
#melt the data into long form
diversity = diversity.melt(id_vars=['Location'],value_name="PercentWhite",var_name="Year")
#print the shape as a sanity check, should have 714 rows
print(diversity.shape)
#fix the year column by only looking at the stuff before __
diversity['Year'] = diversity['Year'].replace(r'(.*)__.*', r'\1', regex=True)
#print out the data frame to ensure correctness
diversity

(714, 3)


,Location,Year,PercentWhite
0,Alabama,2008,0.68670
1,Alaska,2008,0.66160
2,Arizona,2008,0.58220
3,Arkansas,2008,0.75710
4,California,2008,0.42030
...,...,...,...
709,Virginia,2022,0.58904
710,Washington,2022,0.63597
711,West Virginia,2022,0.90436
712,Wisconsin,2022,0.79052


In [6]:
#read in the csv, ignoring rows for headers and the US
age = pd.read_csv('data/age.csv', thousands=",", skiprows=[0,1,3])
#make a list of years we need to find averages for, excluding 2020
list_years = np.arange(2008,2020)
list_years = np.append(list_years, [2021,2022])
#for each value in the list
for i in list_years:
    #make a new column based on the year and populate it with an estimate of the sums of ages
    age[str(i)] = ((age[str(i)+'__Children 0-18']*9) + (age[str(i)+'__Adults 19-25']*22) + (age[str(i)+'__Adults 26-34']*30)
                                  + (age[str(i)+'__Adults 35-54']*45) + (age[str(i)+'__Adults 55-64']*60) + (age[str(i)+'__65+']*75))
    #divide the sum of the ages by the total number of people to find an average
    age[str(i)] = (age[str(i)]) / (age[str(i)+'__Total'])
    
#only select certain columns with the averages
age = age.loc[:,['Location','2008','2010','2012','2014',
                  '2016','2018','2022']]
#melt the dataframe into long format
age = age.melt(id_vars=['Location'],value_name="AverageAge",var_name="Year")
#remove puerto rico from the dataset, since we don't have voting data from there, so we only need 51 states (D.C.)
age = age[age['Location'] != 'Puerto Rico']
#reset the index
age = age.reset_index(drop=True)
#print the shape of the dataframe, should have 357 rows
print(age.shape)
#print the dataframe to verify accuracy
age

(357, 3)


,Location,Year,AverageAge
0,Alabama,2008,37.655371
1,Alaska,2008,34.414999
2,Arizona,2008,36.578626
3,Arkansas,2008,37.745749
4,California,2008,35.884491
...,...,...,...
352,Virginia,2022,40.044919
353,Washington,2022,39.907804
354,West Virginia,2022,42.397487
355,Wisconsin,2022,40.930317


In [7]:
#read in the population data
population = pd.read_csv('data/population.csv', thousands=",")
#melt the data into long form
population = population.melt(id_vars=['Name'],value_name="Population",var_name="Year")
#print the shape for sanity, should have 612 rows
print(population.shape)

#read in the area data
area = pd.read_csv('data/area.csv', thousands=",")

#make a new dataframe with the population and area for each state in each year
population_density = duckdb.sql("""SELECT Name,Year,Population,LandAreaSQM 
                                FROM population LEFT JOIN area ON population.Name = area.State""").df()

#make a new column based on the population density calculation (people/sqmi)
population_density['PopulationDensity'] = 1000*population_density['Population']/population_density['LandAreaSQM']
#print the dataframe to make sure its accurate
population_density

(612, 3)


,Name,Year,Population,LandAreaSQM,PopulationDensity
0,Alaska,2000,627.963,570641,1.100452
1,Arizona,2000,5160.590,113594,45.430128
2,Arkansas,2000,2678.590,52035,51.476698
3,California,2000,33987.980,155779,218.180756
4,Colorado,2000,4326.920,103642,41.748712
...,...,...,...,...,...
607,Maryland,2018,6042.150,9707,622.452869
608,Alabama,2020,5031.860,50645,99.355514
609,Maryland,2020,6173.690,9707,636.003915
610,Alabama,2022,5073.900,50645,100.185606


In [8]:
#merging everything together
#left join election_year_data with education
#note the syntax since we don't want duplicate columns for State,Name 
# #since they would multiple w/ multiple joins
election_demographics = duckdb.sql("""SELECT election_year_data.*, PercentBachelors
                                   FROM election_year_data
                                   LEFT JOIN education ON election_year_data.State = education.Name 
                                   AND election_year_data.Year = education.Year""").df()
#left join election_demographics with income
election_demographics = duckdb.sql("""SELECT election_demographics.*, Income
                                   FROM election_demographics
                                   LEFT JOIN income ON election_demographics.State = income.Name 
                                   AND election_demographics.Year = income.Year""").df()
#left join election_demographics with diversity
election_demographics = duckdb.sql("""SELECT election_demographics.*, PercentWhite
                                   FROM election_demographics
                                   LEFT JOIN diversity 
                                   ON election_demographics.State = diversity.Location 
                                   AND election_demographics.Year = diversity.Year""").df()
#left join election_demographics with age
election_demographics = duckdb.sql("""SELECT election_demographics.*, AverageAge
                                   FROM election_demographics
                                   LEFT JOIN age ON election_demographics.State = age.Location 
                                   AND election_demographics.Year = age.Year""").df()
#left join election_demographics with population_density
election_demographics = duckdb.sql("""SELECT election_demographics.*, 
                                   Population, LandAreaSQM, PopulationDensity
                                   FROM election_demographics
                                   LEFT JOIN population_density 
                                   ON election_demographics.State = population_density.Name 
                                   AND election_demographics.Year = population_density.Year""").df()
#verify that the number of NAs is appropriate 
# (since some entire years are missing, thus might appear too high, but there are still plenty of real values)
print(election_demographics.isna().sum())

#add a presidential election column - T if presidential, F if not
election_demographics['Presidential'] = (election_demographics['Year'] % 4) == 0

#write it to a csv as a backup
# election_demographics.to_csv('data/election_demographics.csv')

#print the data to verify accuracy
election_demographics

State                           0
Year                            0
TotalBallots                   62
VotingEligiblePopulation        0
VotingAgePopulation             0
PercentVotingEligibleVotes     62
PercentVotingAgeVotes          62
PercentBachelors              153
Income                          0
PercentWhite                  255
AverageAge                    255
Population                      0
LandAreaSQM                     0
PopulationDensity               0
dtype: int64


,State,Year,TotalBallots,VotingEligiblePopulation,VotingAgePopulation,PercentVotingEligibleVotes,PercentVotingAgeVotes,PercentBachelors,Income,PercentWhite,AverageAge,Population,LandAreaSQM,PopulationDensity,Presidential
0,Alabama,2008,2105622.0,3454510,3595708,0.609528,0.585593,22.0,60810,0.68670,37.655371,4718.210,50645,93.162405,True
1,Alaska,2008,327341.0,479429,507645,0.682773,0.644823,27.3,87500,0.66160,34.414999,687.455,570641,1.204707,True
2,Arkansas,2008,1095958.0,2071563,2174846,0.529049,0.503924,18.8,54130,0.75710,37.745749,2874.550,52035,55.242625,True
3,Colorado,2008,2422236.0,3382959,3708955,0.716011,0.653078,35.6,83330,0.71060,36.314987,4889.730,103642,47.179039,True
4,Delaware,2008,413562.0,628200,680493,0.658329,0.607739,27.5,69330,0.67740,37.938640,883.874,1949,453.501283,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,Washington,2020,4116894.0,5405804,6034791,0.761569,0.682193,38.4,95180,NaN,NaN,7724.570,66456,116.235855,True
608,Arkansas,2012,1078548.0,2109847,2242740,0.511197,0.480906,21.0,50280,0.74210,38.335746,2952.880,52035,56.747958,True
609,New Jersey,2014,1955042.0,6014127,6940888,0.325075,0.281670,37.4,81900,0.56750,39.047098,8867.280,7354,1205.776448,False
610,Colorado,2022,2540666.0,4357176,4642930,0.583099,0.547212,45.9,93510,0.64987,39.363628,5841.040,103642,56.357847,False
